# Prompt-Management in LangChain

In diesem Notebook lernen wir die wichtigsten Aspekte des Prompt-Managements kennen:
- Organisation und Versionierung von Prompts
- LangChain Hub für Prompt-Sharing
- Integration mit Langfuse für Monitoring und Tracking
- Best Practices für Prompt-Management in größeren Teams

## Vorbereitung: Benötigte Bibliotheken installieren

Falls noch nicht installiert, müssen wir die benötigten Bibliotheken installieren:

In [1]:
# Installation der benötigten Pakete (bei Bedarf ausführen)
# !pip install langchain langchain-openai langfuse langchain-langfuse

In [2]:
# Importieren der benötigten Bibliotheken
import os
from datetime import datetime
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain import hub
import json
from langchain.schema import StrOutputParser

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

# Optional: Für Langfuse-Integration
# from langfuse import Langfuse
# from langfuse.client import StatelessTracer
# from langchain_langfuse import LangfuseCallbackHandler

## 1. Grundlagen: Organisiertes Prompt-Management

Effektives Prompt-Management umfasst folgende Aspekte:
- Strukturierte Speicherung von Prompts
- Versionierung für Nachvollziehbarkeit
- Wiederverwendbarkeit durch modulare Gestaltung
- Testing und Evaluation

Wir beginnen mit einfachen Methoden zur strukturierten Verwaltung von Prompts:

In [3]:
# Lokale Prompt-Verwaltung mit einfacher Versionierung
class PromptManager:
    def __init__(self, base_path="./prompts"):
        self.base_path = base_path
        # Erstelle den Verzeichnispfad, falls er nicht existiert
        os.makedirs(base_path, exist_ok=True)

    def save_prompt(self, prompt, name, version=None, metadata=None):
        # Generiere einen Zeitstempel als Version, falls keine angegeben wurde
        if version is None:
            version = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Erstelle das Verzeichnis für diesen Prompt, falls es nicht existiert
        prompt_dir = os.path.join(self.base_path, name)
        os.makedirs(prompt_dir, exist_ok=True)

        # Speichere den Prompt und Metadaten
        prompt_data = {
            "content": prompt.to_json() if hasattr(prompt, "to_json") else str(prompt),
            "version": version,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {}
        }

        # Speichere als JSON-Datei
        file_path = os.path.join(prompt_dir, f"{version}.json")
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(prompt_data, f, ensure_ascii=False, indent=2)

        return file_path

    def list_versions(self, name):
        prompt_dir = os.path.join(self.base_path, name)
        if not os.path.exists(prompt_dir):
            return []

        # Liste alle JSON-Dateien im Verzeichnis
        versions = [f.replace(".json", "") for f in os.listdir(prompt_dir) if f.endswith(".json")]
        return sorted(versions)

    def get_latest_version(self, name):
        versions = self.list_versions(name)
        if not versions:
            return None
        return versions[-1]

In [4]:
# Beispiel: Erstellen und Speichern eines Prompts
prompt_manager = PromptManager()

# Einen einfachen Prompt erstellen
kundenservice_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Kundendienstmitarbeiter für {unternehmen}. "
             "Du sollst immer freundlich, professionell und lösungsorientiert antworten."),
    ("human", "{kundenfrage}")
])

# Prompt speichern mit Metadaten
prompt_path = prompt_manager.save_prompt(
    prompt=kundenservice_prompt,
    name="kundenservice_basic",
    metadata={
        "autor": "Workshop-Team",
        "verwendung": "Kundenservice-Chatbot",
        "empfohlenes_modell": "gpt-3.5-turbo"
    }
)

print(f"Prompt gespeichert unter: {prompt_path}")
print(f"Verfügbare Versionen: {prompt_manager.list_versions('kundenservice_basic')}")

TypeError: Object of type SystemMessagePromptTemplate is not JSON serializable

## 2. LangChain Hub für Prompt-Sharing

Der LangChain Hub ist eine zentrale Plattform zur Verwaltung und gemeinsamen Nutzung von Prompts. Er bietet folgende Vorteile:
- Zentrales Repository für Prompts
- Versionskontrolle
- Einfache Integration in LangChain-Anwendungen
- Zusammenarbeit im Team

In [5]:
# Beispiel: Prompt aus dem LangChain Hub laden
sentiment_prompt = hub.pull("borislove/customer-sentiment-analysis")

print("Geladener Prompt aus dem Hub:")
print(sentiment_prompt)

/opt/tljh/user/lib/python3.10/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Geladener Prompt aus dem Hub:
input_variables=['client_letter', 'format_instructions'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'borislove', 'lc_hub_repo': 'customer-sentiment-analysis', 'lc_hub_commit_hash': '45eafd27466db2b27a448e29132a77a88a53092b7d4ad09838d13ef23908c36f'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="As a customer service representative, you receive the following email from a customer.\nYour task is to identify the customer's sentiment and categorize it based on the scale below:\n    0 - Calm: Customer asks questions but does not seem upset; is just seeking information.\n    1 - Slightly Frustrated: Customer shows subtle signs of irritation but is still open to solutions.\n    2 - Frustrated: Customer explicitly states being unhappy or irritated but is willing to discuss a solution.\n    3 - Very Frustrated: Customer is clearly agitated, uses strong language, or m

In [6]:
# Beispiel für die Verwendung des geladenen Prompts
client_letter = """Ich bin von dem Produkt zutiefst enttäuscht. Nach nur zwei Wochen Nutzung ist die Verarbeitung bereits mangelhaft. Mehrere Knöpfe sind abgefallen und die Naht hat sich gelöst. Ich erwarte eine vollständige Rückerstattung und werde das Produkt definitiv nicht weiterempfehlen."""

format_instructions = """Zusätzlich zur numerischen Klassifizierung sollst du eine kurze Begründung (max. 50 Wörter) für deine Einschätzung geben. Formatiere deine Antwort wie folgt:
Sentiment: [positiv/negativ/neutral]
Score: [Zahl zwischen 1-10]
Begründung: [Deine Erklärung]"""

# Formatierter Prompt
formatted_prompt = sentiment_prompt.format(
    client_letter=client_letter,
    format_instructions=format_instructions
)

print("\nFormatierter Prompt:")
print(formatted_prompt)


Formatierter Prompt:
System: As a customer service representative, you receive the following email from a customer.
Your task is to identify the customer's sentiment and categorize it based on the scale below:
    0 - Calm: Customer asks questions but does not seem upset; is just seeking information.
    1 - Slightly Frustrated: Customer shows subtle signs of irritation but is still open to solutions.
    2 - Frustrated: Customer explicitly states being unhappy or irritated but is willing to discuss a solution.
    3 - Very Frustrated: Customer is clearly agitated, uses strong language, or mentions the problem repeatedly.
    4 - Extremely Frustrated: Customer is intensely unhappy, may raise their voice or use aggressive language.
    5 - Overwhelmed: Customer seems emotionally upset, says things like 'I can't take this anymore' or 'This is the worst experience ever.'
If you cannot identify the sentiment for some reason, simply respond with 'Unknown'
Human: Letter: 
'''Ich bin von dem

In [7]:
# Prompt ausführen mit LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
sentiment_chain = sentiment_prompt | llm | StrOutputParser()

result = sentiment_chain.invoke({
    "client_letter": client_letter,
    "format_instructions": format_instructions
})

print("\nLLM-Antwort:")
print(result)


LLM-Antwort:
Sentiment: Negativ
Score: 4
Begründung: Der Kunde ist sehr frustriert und enttäuscht über die mangelhafte Qualität des Produkts nach nur kurzer Nutzungsdauer. Er fordert eine Rückerstattung und gibt an, das Produkt nicht weiterzuempfehlen. Die Verwendung von starken Worten wie "zutiefst enttäuscht" deutet auf eine hohe Frustrationsebene hin.


### Eigenen Prompt im Hub veröffentlichen

Um einen eigenen Prompt im LangChain Hub zu veröffentlichen, benötigen Sie einen API-Schlüssel. Hier ist der Prozess:

In [8]:
# Beispiel: Eigenen Prompt erstellen und auf den Hub hochladen
# Hinweis: Erfordert einen API-Schlüssel von LangChain

# 1. Erstellen eines eigenen Prompts
product_review_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Experte für Produktbewertungen. "
              "Analysiere die folgende Produktbewertung und extrahiere wichtige Informationen."),
    ("human", "{review}")
])

# 2. Hochladen zum Hub (nur ausführen, wenn Sie einen API-Schlüssel haben)
# hub.push("mein_username/product-review-analysis", product_review_prompt)
# print("Prompt erfolgreich auf den Hub hochgeladen!")

# Alternativ: Anzeigen, wie der Prompt aussieht
print(product_review_prompt.to_string())

AttributeError: 'ChatPromptTemplate' object has no attribute 'to_string'

## 3. Integration mit Langfuse für Monitoring und Tracking

Langfuse ist ein Observability-Tool für LLM-Anwendungen, das Monitoring, Tracing und Evaluation von Prompt-Ausführungen ermöglicht. Die Integration mit LangChain ermöglicht die Überwachung und Optimierung von Prompts im Produktionseinsatz.

In [9]:
# Langfuse-Integration (Kommentiert, da API-Schlüssel erforderlich)
"""
# Langfuse initialisieren
langfuse = Langfuse(
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY"),
    host=os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com")
)

# LangChain-Callback-Handler für Langfuse
handler = LangfuseCallbackHandler(
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY")
)

# LLM mit Langfuse-Integration
llm_with_tracing = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    callbacks=[handler]
)

# Prompt mit Tracing ausführen
chain_with_tracing = product_review_prompt | llm_with_tracing | StrOutputParser()
result = chain_with_tracing.invoke({"review": "Ein wirklich tolles Produkt! Sehr empfehlenswert."})
"""

# Stattdessen zeigen wir, wie Langfuse in einem realen Projekt funktionieren würde
print("--- Langfuse Integration Demo ---")
print("1. Langfuse erstellt automatisch Traces für alle LLM-Aufrufe")
print("2. Jede Prompt-Ausführung wird mit Metadaten (Modell, Tokens, Latenz, Kosten) protokolliert")
print("3. Sie können Prompts basierend auf Performance-Metriken optimieren")
print("4. Feedback-Schleifen ermöglichen kontinuierliche Verbesserung")

--- Langfuse Integration Demo ---
1. Langfuse erstellt automatisch Traces für alle LLM-Aufrufe
2. Jede Prompt-Ausführung wird mit Metadaten (Modell, Tokens, Latenz, Kosten) protokolliert
3. Sie können Prompts basierend auf Performance-Metriken optimieren
4. Feedback-Schleifen ermöglichen kontinuierliche Verbesserung


### Beispiel: Visualisierung von Langfuse-Daten

In der Langfuse-Oberfläche können Sie folgende Informationen einsehen:

- Vollständige Traces aller LLM-Interaktionen
- Prompt-Performance (Latenz, Token-Nutzung, Kosten)
- Modellvergleiche für verschiedene Prompts
- Versionierungseffekte auf die Performance

![Langfuse Dashboard Beispiel](https://docs.langfuse.com/img/observability/dashboard.png)

*(Bild: Beispiel eines Langfuse Dashboards, Quelle: Langfuse Dokumentation)*

## 4. Best Practices für Prompt-Management in Teams

### Strukturierung und Namenskonventionen

Eine konsistente Struktur für Prompts erleichtert die Wartung und Entwicklung:

In [10]:
# Beispiel für strukturierte Prompt-Entwicklung

# 1. System-Rolle definieren (separat für bessere Wiederverwendbarkeit)
customer_service_system = """Du bist ein{gender} freundliche{gender} und lösungsorientierte{gender}
Kundendienstmitarbeiter{gender} von {company_name}.
Du hilfst Kunden bei Problemen mit {product_category}.
Wichtige Unternehmensrichtlinien:
- {policy1}
- {policy2}
"""

# 2. Beispiele als Demonstration des gewünschten Verhaltens
examples = [
    {"question": "Mein Produkt funktioniert nicht mehr",
     "answer": "Es tut mir leid zu hören, dass Sie Probleme mit Ihrem Produkt haben. "
               "Können Sie mir bitte das genaue Modell und den Fehler beschreiben, "
               "damit ich Ihnen besser helfen kann?"},
    # Weitere Beispiele...
]

# 3. Modularer Prompt-Aufbau mit ChatPromptTemplate
modular_prompt = ChatPromptTemplate.from_messages([
    ("system", customer_service_system),
    ("human", "Hier sind einige Beispiele für gute Antworten: {examples}"),
    ("human", "{customer_query}")
])

# 4. Verwendung mit konkreten Werten
formatted_modular_prompt = modular_prompt.format(
    gender="r",  # für weibliche Form
    company_name="TechSupport GmbH",
    product_category="Elektronikgeräten",
    policy1="Rückgabe innerhalb von 14 Tagen ohne Angabe von Gründen möglich",
    policy2="Bei Defekten innerhalb der Garantiezeit erfolgt kostenloser Ersatz",
    examples=str(examples),  # Vereinfachte Darstellung für das Beispiel
    customer_query="Wie kann ich mein defektes Smartphone zurücksenden?"
)

print("Formatierter modularer Prompt:")
print(formatted_modular_prompt)

Formatierter modularer Prompt:
System: Du bist einr freundlicher und lösungsorientierter 
Kundendienstmitarbeiterr von TechSupport GmbH. 
Du hilfst Kunden bei Problemen mit Elektronikgeräten.
Wichtige Unternehmensrichtlinien:
- Rückgabe innerhalb von 14 Tagen ohne Angabe von Gründen möglich
- Bei Defekten innerhalb der Garantiezeit erfolgt kostenloser Ersatz

Human: Hier sind einige Beispiele für gute Antworten: [{'question': 'Mein Produkt funktioniert nicht mehr', 'answer': 'Es tut mir leid zu hören, dass Sie Probleme mit Ihrem Produkt haben. Können Sie mir bitte das genaue Modell und den Fehler beschreiben, damit ich Ihnen besser helfen kann?'}]
Human: Wie kann ich mein defektes Smartphone zurücksenden?


### Versionierungs-Workflow in Teams

Ein effektiver Workflow für Prompt-Entwicklung im Team umfasst diese Schritte:

1. **Entwicklung**: Lokale Iteration und Testing
2. **Review**: Peer-Review von Prompt-Änderungen
3. **Testing**: Automatisierte Tests für konsistente Ergebnisse
4. **Staging**: Testen in einer produktionsähnlichen Umgebung
5. **Produktion**: Deployment mit Monitoring
6. **Evaluation**: Kontinuierliche Bewertung und Feedback-Sammlung

Hier ist ein Beispiel für ein einfaches Testframework:

In [11]:
# Einfaches Test-Framework für Prompts

def test_prompt(prompt_template, test_cases, llm=None):
    """Test eines Prompts mit mehreren Testfällen"""
    if llm is None:
        llm = ChatOpenAI(temperature=0)  # Deterministisches Verhalten für Tests

    results = []
    chain = prompt_template | llm | StrOutputParser()

    for i, test_case in enumerate(test_cases):
        try:
            # Prompt mit Testdaten ausführen
            response = chain.invoke(test_case["input"])

            # Einfache Validierung (kann mit spezifischeren Prüfungen erweitert werden)
            validation = {
                "passed": True,
                "notes": "Test bestanden"
            }

            # Wenn expected_contains definiert ist, prüfen ob es in der Antwort enthalten ist
            if "expected_contains" in test_case:
                for expected in test_case["expected_contains"]:
                    if expected.lower() not in response.lower():
                        validation["passed"] = False
                        validation["notes"] = f"Erwarteter Text nicht gefunden: {expected}"

            # Wenn expected_format definiert ist (z.B. JSON), Format überprüfen
            if "expected_format" in test_case and test_case["expected_format"] == "json":
                try:
                    json.loads(response)
                except json.JSONDecodeError:
                    validation["passed"] = False
                    validation["notes"] = "Antwort ist kein gültiges JSON"

            results.append({
                "test_case": i + 1,
                "input": test_case["input"],
                "response": response,
                "validation": validation
            })
        except Exception as e:
            results.append({
                "test_case": i + 1,
                "input": test_case["input"],
                "error": str(e),
                "validation": {"passed": False, "notes": f"Fehler bei der Ausführung: {e}"}
            })

    return results

In [12]:
# Beispiel: Prompt-Tests durchführen

# Einfacher Testprompt für Produktbeschreibungen
product_desc_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Produktbeschreibungs-Generator für einen Online-Shop. "
              "Erstelle eine kurze, überzeugende Beschreibung für das folgende Produkt."),
    ("human", "Produkt: {product_name}\nKategorie: {category}\nHauptmerkmale: {features}")
])

# Testfälle definieren
test_cases = [
    {
        "input": {
            "product_name": "Wireless Kopfhörer XZ-500",
            "category": "Audio",
            "features": "Bluetooth 5.0, 30h Akkulaufzeit, Noise-Cancelling"
        },
        "expected_contains": ["Kopfhörer", "Akku", "Noise"]
    },
    {
        "input": {
            "product_name": "Ergonomische Bürotastatur K-2000",
            "category": "Computer-Zubehör",
            "features": "Mechanische Switches, RGB-Beleuchtung, Handballenauflage"
        },
        "expected_contains": ["Tastatur", "ergonomisch"]
    }
]

# Tests ausführen
test_results = test_prompt(product_desc_prompt, test_cases)

# Ergebnisse anzeigen
for result in test_results:
    print(f"\nTestfall {result['test_case']}:")
    print(f"Status: {'✅ Bestanden' if result['validation']['passed'] else '❌ Fehlgeschlagen'}")
    print(f"Notizen: {result['validation']['notes']}")
    print(f"Antwort: {result['response'][:150]}...")


Testfall 1:
Status: ✅ Bestanden
Notizen: Test bestanden
Antwort: Tauche ein in deine Lieblingsmusik mit den Wireless Kopfhörern XZ-500! Dank Bluetooth 5.0 Technologie genießt du kabellose Freiheit und mit einer Akku...

Testfall 2:
Status: ✅ Bestanden
Notizen: Test bestanden
Antwort: Entdecke die ultimative Ergonomie und Leistung mit der ergonomischen Bürotastatur K-2000! Mit mechanischen Switches für ein präzises Tippgefühl, RGB-B...


## 5. Zusammenfassung: Best Practices für Prompt-Management

1. **Strukturierung und Modularität**
   - Prompts in wiederverwendbare Komponenten aufteilen
   - Klare Namenskonventionen verwenden
   - Metadaten für Kontext und Nutzung hinzufügen

2. **Versionierung**
   - Jede Version eindeutig identifizierbar machen
   - Änderungshistorie dokumentieren
   - A/B-Testing für neue Versionen durchführen

3. **Zusammenarbeit**
   - Zentrale Repositories wie LangChain Hub nutzen
   - Review-Prozesse implementieren
   - Dokumentation für jede Prompt-Familie führen

4. **Testing und Evaluation**
   - Automatisierte Tests mit erwarteten Ergebnissen
   - Unterschiedliche Eingabeszenarien abdecken
   - Edge Cases berücksichtigen

5. **Monitoring und Optimierung**
   - Tools wie Langfuse für Observability einsetzen
   - Performance-Metriken verfolgen (Latenz, Token, Kosten)
   - Nutzer-Feedback systematisch erfassen und einbeziehen

## Übungsaufgaben

1. Erstellen Sie einen parametrisierten Prompt für einen bestimmten Anwendungsfall (z.B. Produkt-Rezensionen, Support-Anfragen)
2. Implementieren Sie eine lokale Prompt-Versionierung mit dem PromptManager
3. Testen Sie Ihren Prompt mit verschiedenen Eingabeszenarien
4. Erstellen Sie eine verbesserte Version Ihres Prompts und vergleichen Sie die Ergebnisse
5. (Optional) Veröffentlichen Sie Ihren Prompt im LangChain Hub